# Initial requirements

This notebook requires IBM Cloud Object Storage and IBM Cloud Functions
Please follow IBM Cloud dashboard and create both services.


In [ ]:
# !pip --version

In [ ]:
#Install PyWren-IBM
!{sys.executable} -m pip install -U pywren-ibm-cloud

import pywren_ibm_cloud as pywren
pywren.__version__

In [ ]:
# We need this to overcome Python notebooks limitations of too many open files
import resource
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('Bebore:', soft, hard)

# Raising the soft limit. Hard limits can be raised only by sudo users
resource.setrlimit(resource.RLIMIT_NOFILE, (10000, hard))
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('After:', soft, hard)

In [ ]:
%config Completer.use_jedi = False
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt
from scipy.sparse import coo_matrix
from collections import defaultdict
from pyImagingMSpec.image_measures import isotope_image_correlation, isotope_pattern_match
from cpyImagingMSpec import measure_of_chaos
from itertools import chain
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
import sys
import io

### IBM COS Setup

Setup a bucket in IBM Cloud Object Storage
You need an IBM COS bucket which you will use to store the input data. If you don't know of any of your existing buckets or would like like to create a new one, please navigate to your cloud resource list, then find and select your storage instance. From here, you will be able to view all your buckets and can create a new bucket in the region you prefer. Make sure you copy the correct endpoint for the bucket from the Endpoint tab of this COS service dashboard. Note: The bucket names must be unique.

In [ ]:
# Fill here the bucket name you created in COS Dashboard 
#bucket_name = 'kovalev-annotation-example'

In [ ]:
# Define COS endpoint information. Example of US Cross Region endpoint
#cos_endpoint = 's3.private.eu-de.cloud-object-storage.appdomain.cloud'

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

Obtain the API key and endpoint to the IBM Cloud Functions service. Navigate to Getting Started > API Key from the side menu and copy the values for "Current Namespace", "Host" and "Key" into the config below. Make sure to add "https://" to the host when adding it as the endpoint.

In [ ]:
config = {
    'ibm_cf':  {'endpoint': 'https://eu-gb.functions.cloud.ibm.com',
              'namespace': 'kovalev@embl.de_dev', 
              'action_memory' : 1024,
              'api_key': '***'}, 
    'ibm_cos': {
        'endpoint': 'https://s3.eu-gb.cloud-object-storage.appdomain.cloud',
        'api_key' : '***'
    },
    'pywren' : {'storage_bucket' : 'metaspace-annotation-example'}
}

In [ ]:
import json

secrets = json.load(open('config.json'))
config['ibm_cf']['api_key'] = secrets['ibm_cf']['api_key']
config['ibm_cos']['api_key'] = secrets['ibm_cos']['api_key']

### IBM Cloud Functions setup

The PyWren engine requires its server side component to be deployed in advance. This step creates a new IBM Cloud Functions function with the PyWren server side runtime. This action will be used internally by PyWren during execution phases.

In [ ]:
pywren.runtime.clone_runtime('ibmfunctions/pywren-metabolomics:3.6', config)

In [ ]:
pw = pywren.ibm_cf_executor(config=config, runtime='pywren-metabolomics_3.6')

# Upload test data into COS bucket

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config
from ibm_botocore.client import ClientError

In [ ]:
cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config['ibm_cos']['api_key'],
#                               ibm_auth_endpoint=config['ibm_cos']['auth_endpoint'],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config['ibm_cos']['endpoint'])

In [ ]:
def copy(src, target_bucket, target_key):
    print('Copying from {} to {}/{}'.format(src, target_bucket, target_key))

    with open(src, "rb") as fp:
        cos_client.put_object(Bucket=target_bucket, Key=target_key, Body=fp)

    print('Copy completed for {}/{}'.format(target_bucket, target_key))

In [ ]:
import os

for dirpath, dirnames, filenames in os.walk('./metabolomics'):
    for fn in filenames:
        f_path = f'{dirpath}/{fn}'
        copy(f_path, config['pywren']['storage_bucket'], f_path)

In [ ]:
input_data =   {'ds': 'metabolomics/input/ds.txt',
                'ds_coord': 'metabolomics/input/ds_coord.txt',
                'segments': 'metabolomics/input/segments'}
input_db =     {'centroids': 'metabolomics/db/centroids.csv',
                'formulas': 'metabolomics/db/formulas.csv'}
input_config = {'config': 'metabolomics/data/config.json',
                'meta': 'metabolomics/data/meta.json'}

# Read Dataset Spectra

In [ ]:
def parse_txt(key, data_stream, func):
    rows = []
    buffer = io.StringIO(data_stream.read().decode('utf-8'))
    while True:
        line = buffer.readline()
        if not line:
            break
        rows.append(func(line))
    return rows

In [ ]:
def parse_spectrum_line(s):
    ind_s, mzs_s, int_s = s.split('|')
    return (int(ind_s),
            np.fromstring(mzs_s, sep=' ').astype('float32'),
            np.fromstring(int_s, sep=' '))

In [ ]:
def parse_spectrum_coord(s):
    sp_i, x, y = map(int, s.split(','))
    return (sp_i, x, y)

In [ ]:
bucket_name = config['pywren']['storage_bucket']
bucket_name

In [ ]:
iterdata = [[f'{bucket_name}/{input_data["ds"]}', parse_spectrum_line]]
# NOTE: we need to be absolutely sure that using chunk_size doesn't split a line into separate chunks
futures = pw.map(parse_txt, iterdata, chunk_size=64*1024**2)
results = pw.get_result(futures)
spectra = []
for res_list in results:
    spectra.extend(res_list)

In [ ]:
len(spectra)

In [ ]:
sp_i, mzs, ints = spectra[0]
mzs

In [ ]:
ints

In [ ]:
iterdata = [[f'{bucket_name}/{input_data["ds_coord"]}', parse_spectrum_coord]]
futures = pw.map(parse_txt, iterdata)
spectra_coords = pw.get_result(futures)

In [ ]:
len(spectra_coords)

In [ ]:
spectra_coords[:5]

# Read Molecular Database

In [ ]:
def process_formulas_database(key, data_stream):
    formulas_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    return formulas_df.shape, formulas_df.head()

In [ ]:
iterdata = [f'{bucket_name}/{input_db["formulas"]}']
futures = pw.map(process_formulas_database, iterdata)
formulas_shape, formulas_head = pw.get_result(futures)

In [ ]:
formulas_shape

In [ ]:
formulas_head

In [ ]:
def process_centroids_database(key, data_stream):
    centroids_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    return centroids_df.shape, centroids_df.head(8)

In [ ]:
%%time
iterdata = [f'{bucket_name}/{input_db["centroids"]}']
futures = pw.map(process_centroids_database, iterdata)
centroids_shape, centroids_head = pw.get_result(futures)

In [ ]:
centroids_shape

In [ ]:
centroids_head

# Generate Ion Images

In [ ]:
def real_pixel_indices(spectra_coords):
    coord_pairs = [r[1:] for r in spectra_coords]

    min_x, min_y = np.amin(np.asarray(coord_pairs), axis=0)
    max_x, max_y = np.amax(np.asarray(coord_pairs), axis=0)

    _coord = np.array(coord_pairs)
    _coord = np.around(_coord, 5)  # correct for numerical precision
    _coord -= np.amin(_coord, axis=0)

    nrows, ncols = (max_y - min_y + 1,
                    max_x - min_x + 1)

    pixel_indices = _coord[:, 1] * ncols + _coord[:, 0]
    return pixel_indices.astype(np.int32), nrows, ncols

In [ ]:
def sp_df_gen(sp_it, pixel_indices):
    for sp_id, mzs, intensities in sp_it:
        for mz, ints in zip(mzs, intensities):
            yield pixel_indices[sp_id], mz, ints

In [ ]:
def gen_iso_images(spectra_it, pixel_indices, centr_df, nrows, ncols, ppm, min_px=1):
    if len(centr_df) > 0:
        # a bit slower than using pure numpy arrays but much shorter
        # may leak memory because of https://github.com/pydata/pandas/issues/2659 or smth else
        sp_df = pd.DataFrame(sp_df_gen(spectra_it, pixel_indices),
                             columns=['idx', 'mz', 'ints']).sort_values(by='mz')

        # -1, + 1 are needed to extend sf_peak_mz range so that it covers 100% of spectra
        centr_df = centr_df[(centr_df.mz >= sp_df.mz.min() - 1) &
                            (centr_df.mz <= sp_df.mz.max() + 1)]
        lower = centr_df.mz.map(lambda mz: mz - mz * ppm * 1e-6)
        upper = centr_df.mz.map(lambda mz: mz + mz * ppm * 1e-6)
        lower_idx = np.searchsorted(sp_df.mz, lower, 'l')
        upper_idx = np.searchsorted(sp_df.mz, upper, 'r')

        for i, (l, u) in enumerate(zip(lower_idx, upper_idx)):
            if u - l >= min_px:
                data = sp_df.ints[l:u].values
                if data.shape[0] > 0:
                    idx = sp_df.idx[l:u].values
                    row_inds = idx / ncols
                    col_inds = idx % ncols
                    m = coo_matrix((data, (row_inds, col_inds)), shape=(nrows, ncols), copy=True)
                    yield centr_df.index[i], (centr_df.peak_i.iloc[i], m)

In [ ]:
def merge_formula_images(iso_images):
    formula_images = defaultdict(list)
    for formula_i, (peak_i, img) in iso_images:
        formula_images[formula_i].append((peak_i, img))

    def filter_formula_images():
        filtered_f_images = {}
        for f_i, images in formula_images.items():
            images = sorted(images, key=lambda x: x[0])
            if len(images) > 1 and images[0][0] == 0:
                filtered_f_images[f_i] = images
        return filtered_f_images

    return filter_formula_images()

In [ ]:
pixel_indices, nrows, ncols = real_pixel_indices(spectra_coords)

In [ ]:
pixel_indices

In [ ]:
nrows, ncols

# Score Formula Images

In [ ]:
def score_formula_images(f_images, centroids_df, empty_image):
    formula_scores = []
    for formula_i, images in f_images.items():
        centr_ints = centroids_df.loc[formula_i].int.values

        image_list = [empty_image] * len(centr_ints)
        for peak_i, img in images:
            image_list[peak_i] = img.toarray()
        flat_image_list = [img.flat[:] for img in image_list]

        m1 = isotope_pattern_match(flat_image_list, centr_ints)
        m2 = isotope_image_correlation(flat_image_list, centr_ints[1:])
        m3 = measure_of_chaos(image_list[0], nlevels=30)
        formula_scores.append([formula_i, m1, m2, m3])

    formula_scores_df = pd.DataFrame(formula_scores,
                                     columns=['formula_i', 'm1', 'm2', 'm3']).set_index('formula_i')
    formula_scores_df['msm'] = formula_scores_df.m1 * formula_scores_df.m2 * formula_scores_df.m3
    return formula_scores_df

In [ ]:
empty_image = np.zeros((nrows, ncols))

In [ ]:
empty_image.shape

# Split Dataset into Segments

In [ ]:
segm_n = 256

In [ ]:
def get_segm_intervals(key, data_stream):
    centroids_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    segm_bounds_q = [i * 1 / segm_n for i in range(1, segm_n)]
    segm_bounds = [np.quantile(centroids_df.mz.values, q) for q in segm_bounds_q]
    segm_bounds = [0.] + segm_bounds + [sys.float_info.max]
    segm_intervals = list(zip(segm_bounds[:-1], segm_bounds[1:]))
    return segm_intervals

In [ ]:
iterdata = [f'{bucket_name}/{input_db["centroids"]}']
futures = pw.map(get_segm_intervals, iterdata)
segm_intervals = pw.get_result(futures)

In [ ]:
segm_intervals[:5]

In [ ]:
def iterate_over_segment(spectra, min_mz, max_mz):
    for sp_i, mzs, ints in spectra:
        smask = (mzs >= min_mz) & (mzs <= max_mz)
        yield [sp_i], mzs[smask], ints[smask]

In [ ]:
def store_segm(key, data_stream, ibm_cos, segm_i, interval):
    spectra = parse_txt(key, data_stream, parse_spectrum_line)
    segm_it = iterate_over_segment(spectra, *interval)
    segm_spectra = pickle.dumps(np.array(list(segm_it)))
    ibm_cos.put_object(Bucket=bucket_name,
                       Key=f'{input_data["segments"]}/{segm_i}.pickle',
                       Body=segm_spectra)

In [ ]:
%%time
iterdata = [[f"{bucket_name}/{input_data['ds']}", segm_i, segm_intervals[segm_i]]
             for segm_i in range(segm_n)]
futures = pw.map(store_segm, iterdata)
results = pw.get_result(futures)

# Annotation Pipeline Applied to each Segment in Parallel

In [ ]:
def filter_formula_images(formula_images, formula_scores_df):
    return {f_i: images
            for (f_i, images) in formula_images.items()
            if f_i in formula_scores_df.index}

In [ ]:
def annotate_segm_spectra(key, data_stream, ibm_cos):
    print("Welcome to annotate spectra")
    centroids_stream = ibm_cos.get_object(Bucket=bucket_name, Key=input_db['centroids'])['Body']
    print("Read centroids DB from IBM COS")
    centroids_df = pd.read_csv(centroids_stream).set_index('formula_i')
    print("Create Pandas DF for centroids DB")
    spectra = pickle.loads(data_stream.read())
    iso_images = gen_iso_images(spectra, pixel_indices, centroids_df, nrows, ncols, ppm=3)
    formula_images = merge_formula_images(list(iso_images))
    formula_scores_df = score_formula_images(formula_images, centroids_df, empty_image)
    formula_scores_df = formula_scores_df[formula_scores_df.msm > 0]
    formula_images = filter_formula_images(formula_images, formula_scores_df)
    return formula_scores_df, formula_images 

In [ ]:
%%time
iterdata = [f'{bucket_name}/{input_data["segments"]}/{segm_i}.pickle' for segm_i in range(segm_n)]
futures = pw.map(annotate_segm_spectra, iterdata)
results = pw.get_result(futures)

In [ ]:
len(results)

# Get Results

In [ ]:
formula_scores_list, formula_images_list = list(zip(*results))
formula_scores_df = pd.concat(formula_scores_list)
formula_images = dict(chain(*[segm_formula_images.items()
                              for segm_formula_images in formula_images_list]))

In [ ]:
img = formula_images[896952][0][1]
plt.imshow(img.toarray())

## Clean Segments Datasets

In [ ]:
def clean_segments_datasets(bucket, key, data_stream, ibm_cos):
    ibm_cos.delete_object(Bucket=bucket, Key=key)

In [ ]:
%%time
data_stream = f'{bucket_name}/{input_data["segments"]}'
futures = pw.map(clean_segments_datasets, data_stream)
pw.get_result(futures)

In [ ]:
pw.clean()